# Veteran Population Import and Set Combination
---



In [1]:
# importing necessary libraries
import pandas as pd
import scipy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool, cpu_count #for using multiple processor cores
import os
import time

## PUMs 2018 Data Import
---
**Warning:** Minimum 16GBs of RAM needed and even then that may not be enough to run the entire sheet.

This is a very large dataset.  So large that it comes in multiple peices.  In order, to optimize RAM this sheet will import, and clean each data set seperatley before joining them.

#### Process:

1. Read in data
2. Remove unnecessary columns
3. Clean data
4. Merge dataset
5. Export CSV

In [2]:
# Read in data and timing it
start = time.time() 
pop1 = pd.read_csv("../PUMs_data/psam_pusa.csv")
end = time.time()
print('total time (s)= ' + str(end-start))

total time (s)= 61.71481704711914


In [3]:
# Checking 
pop1.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2018GQ0000049,6,1,1600,3,1,1013097,75,19,...,140,74,73,7,76,75,80,74,7,72
1,P,2018GQ0000058,6,1,1900,3,1,1013097,75,18,...,76,78,7,76,80,78,7,147,150,75
2,P,2018GQ0000219,6,1,2000,3,1,1013097,118,53,...,117,121,123,205,208,218,120,19,123,18
3,P,2018GQ0000246,6,1,2400,3,1,1013097,43,28,...,43,76,79,77,80,44,46,82,81,8
4,P,2018GQ0000251,6,1,2701,3,1,1013097,16,25,...,4,2,29,17,15,28,17,30,15,1


In [4]:
'''Additional viewing options'''
# This code sets the notebook to display maximum columns.  Uncomment it to see trunacted view
pd.set_option('display.max_columns', len(pop1))  

# This resets the view to a truncated output
#pd.reset_option('display.max_rows')   

In [5]:
'''This is designed to remove large amounts of unnecessary data.  In the next sheet there will be
a more fine tuned atttempt at data cleaning.'''
def column_remover(df):
    start = time.time() 
    
    # Remove all individuals under the age of 20
    df = df[df.AGEP >= 20]
    
    # Remove columns do not help answer the question or are too granular for this analysis
    df = df.drop(columns= ['RT', 'FAGEP', 'FANCP', 'NWLA',
                        'SPORDER', 'FCITWP', 'FWAGP', 'LANP',
                        'CITWP', 'FCOWP', 'FDDRSP',
                        'FER', 'FDEARP', 'FDEYEP',
                        'GCL','FDISP', 'FDOUTP', 'NAICSP',
                        'GCM','FDPHYP', 'FDRATP', 'NOP',
                        'GCR', 'FDRATXP', 'FDREMP', 'OCCP',
                        'HINS7', 'FENGP', 'FESRP', 'PRIVCOV',
                        'JWMNP', 'FFERP', 'FFODP', 'PUBCOV',
                        'MARHM', 'FGCLP', 'FGCMP', 'POWPUMA',
                        'NWRE', 'FGCRP', 'FHICOVP', 'QTRBIR',
                        'ANC', 'FHINS1P', 'FHINS2P',
                        'ANC1P', 'FHINS3C', 'FHINS3P',
                        'ANC2P', 'FHINS4C', 'FHINS4P',
                        'ESP', 'FHINS7P', 'FHISP', 'FINDP',
                        'JWAP','FINTP', 'FJWDP', 'FJWMNP',
                        'JWDP','FJWRIP', 'FJWTRP', 'FLANP',
                        'RAC3P', 'FLANXP', 'FMARP', 'FMARHDP',
                        'RC','FMARHDP', 'FMARHMP', 'FMARHTP', 
                        'SFN', 'FMARHWP', 'FMIGP', 'FMIGSP',
                        'SOCP', 'FOCCP', 'FOIP', 'FPAP',
                        'SFR', 'FPERNP', 'FPINCP', 'FPOBP',
                        'FPOWSP', 'FPRIVCOVP', 'FRACP', 'FRELP',
                        'FRETP', 'FSCHGP', 'FSCHLP', 'FSCHP', 
                        'FSEMP', 'FSEXP', 'FSSIP', 'FSSP', 
                        'FWKHP', 'FWKLP', 'FWKWP', 'FWRKP',
                        'FYOEP', 'JWTR', 'MARHYP', 'NWRE',
                        'RELP', 'SCHG', 'DECADE',
                        'DRIVESP', 'FOD1P', 'FOD2P',
                        'MIGPUMA', 'MIGSP', 'PAOC', 'NWAV',
                        'POBP', 'POWSP', 'RAC2P', 'NWAB',
                        'RACPI', 'INDP', 'NOP', 'MIG'
                        ]) 
    
    # The following PWGTP# columns lacked a proper description and are redundant.  This will drop them
    cols = []
    for i in range(1, 81):
        name = 'PWGTP'+str(i)
        cols.insert(int(-1), name)
    df = df.drop(columns=cols)
    
    end = time.time()
    print('total time (s)= ' + str(end-start))
    return df

In [6]:
pop1=column_remover(pop1)
pop1.head()

total time (s)= 5.046984910964966


,SERIALNO,DIVISION,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,CIT,COW,DDRS,DEAR,DEYE,DOUT,DPHY,DRAT,DRATX,DREM,ENG,HINS1,HINS2,HINS3,HINS4,HINS5,HINS6,INTP,JWRIP,LANX,MAR,MARHD,MARHT,MARHW,MIL,MLPA,MLPB,MLPCD,MLPE,MLPFG,MLPH,MLPI,MLPJ,MLPK,NWLK,OIP,PAP,RETP,SCH,SCHL,SEMP,SEX,SSIP,SSP,WAGP,WKHP,WKL,WKW,WRK,YOEP,DIS,ESR,HICOV,HISP,MSP,NATIVITY,OC,PERNP,PINCP,POVPIP,RAC1P,RACAIAN,RACASN,RACBLK,RACNH,RACNUM,RACSOR,RACWHT,SCIENGP,SCIENGRLP,VPS,WAOB,FCITP,FHINS5C,FHINS5P,FHINS6P,FMARHYP,FMILPP,FMILSP,FPUBCOVP
2,2018GQ0000219,6,2000,3,1,1013097,118,53,1,5.0,2.0,2,2,2.0,2.0,NaN,NaN,1.0,2.0,2,2,2,2,2,2,0.0,NaN,1.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,17.0,0.0,1,0.0,0.0,10000.0,40.0,1.0,1.0,1.0,NaN,1,6.0,2,2,6.0,1,NaN,10000.0,10000.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
3,2018GQ0000246,6,2400,3,1,1013097,43,28,1,NaN,2.0,2,2,2.0,2.0,NaN,2.0,2.0,NaN,2,2,2,2,2,2,0.0,NaN,2.0,5,NaN,NaN,NaN,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,19.0,0.0,1,0.0,0.0,0.0,NaN,3.0,NaN,NaN,NaN,2,6.0,2,1,6.0,1,NaN,0.0,0.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,1.0,1,0,NaN,0,0,0,0,0,0
4,2018GQ0000251,6,2701,3,1,1013097,16,25,1,1.0,2.0,2,2,2.0,2.0,NaN,NaN,1.0,NaN,1,2,2,2,2,2,0.0,NaN,2.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,12.0,0.0,2,0.0,0.0,0.0,NaN,2.0,NaN,NaN,NaN,1,6.0,1,1,6.0,1,NaN,0.0,0.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
5,2018GQ0000390,6,2400,3,1,1013097,25,30,1,NaN,1.0,2,2,1.0,1.0,NaN,NaN,1.0,NaN,2,2,2,1,2,2,0.0,NaN,2.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,16.0,0.0,2,0.0,0.0,0.0,NaN,3.0,NaN,NaN,NaN,1,6.0,1,1,6.0,1,NaN,0.0,0.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
6,2018GQ0000510,6,400,3,1,1013097,18,66,1,NaN,1.0,2,2,1.0,1.0,NaN,NaN,1.0,NaN,2,2,1,2,2,2,0.0,NaN,2.0,2,2.0,1.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,19.0,0.0,2,0.0,15000.0,0.0,NaN,3.0,NaN,NaN,NaN,1,6.0,1,1,3.0,1,NaN,0.0,15000.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,1,0,0,0


In [7]:
# Read in set 2 of 2 
start = time.time() 
pop2 = pd.read_csv("../PUMs_data/psam_pusa.csv")
end = time.time()
print('total time (s)= ' + str(end-start))

total time (s)= 36.58467197418213


In [8]:
pop2=column_remover(pop2)
pop2.info()

total time (s)= 5.2655158042907715
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1272871 entries, 2 to 1648511
Data columns (total 88 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   SERIALNO   1272871 non-null  object 
 1   DIVISION   1272871 non-null  int64  
 2   PUMA       1272871 non-null  int64  
 3   REGION     1272871 non-null  int64  
 4   ST         1272871 non-null  int64  
 5   ADJINC     1272871 non-null  int64  
 6   PWGTP      1272871 non-null  int64  
 7   AGEP       1272871 non-null  int64  
 8   CIT        1272871 non-null  int64  
 9   COW        928083 non-null   float64
 10  DDRS       1272871 non-null  float64
 11  DEAR       1272871 non-null  int64  
 12  DEYE       1272871 non-null  int64  
 13  DOUT       1272871 non-null  float64
 14  DPHY       1272871 non-null  float64
 15  DRAT       25134 non-null    float64
 16  DRATX      124536 non-null   float64
 17  DREM       1272871 non-null  float64
 18  ENG    

In [10]:
# Combine Person Data A and B
pop = pop1.append(pop2)
pop.to_csv('../working_data/us_pop_1yr_compressed.csv', index=False)
pop.head(10)

,SERIALNO,DIVISION,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,CIT,COW,DDRS,DEAR,DEYE,DOUT,DPHY,DRAT,DRATX,DREM,ENG,HINS1,HINS2,HINS3,HINS4,HINS5,HINS6,INTP,JWRIP,LANX,MAR,MARHD,MARHT,MARHW,MIL,MLPA,MLPB,MLPCD,MLPE,MLPFG,MLPH,MLPI,MLPJ,MLPK,NWLK,OIP,PAP,RETP,SCH,SCHL,SEMP,SEX,SSIP,SSP,WAGP,WKHP,WKL,WKW,WRK,YOEP,DIS,ESR,HICOV,HISP,MSP,NATIVITY,OC,PERNP,PINCP,POVPIP,RAC1P,RACAIAN,RACASN,RACBLK,RACNH,RACNUM,RACSOR,RACWHT,SCIENGP,SCIENGRLP,VPS,WAOB,FCITP,FHINS5C,FHINS5P,FHINS6P,FMARHYP,FMILPP,FMILSP,FPUBCOVP
2,2018GQ0000219,6,2000,3,1,1013097,118,53,1,5.0,2.0,2,2,2.0,2.0,NaN,NaN,1.0,2.0,2,2,2,2,2,2,0.0,NaN,1.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,17.0,0.0,1,0.0,0.0,10000.0,40.0,1.0,1.0,1.0,NaN,1,6.0,2,2,6.0,1,NaN,10000.0,10000.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
3,2018GQ0000246,6,2400,3,1,1013097,43,28,1,NaN,2.0,2,2,2.0,2.0,NaN,2.0,2.0,NaN,2,2,2,2,2,2,0.0,NaN,2.0,5,NaN,NaN,NaN,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,19.0,0.0,1,0.0,0.0,0.0,NaN,3.0,NaN,NaN,NaN,2,6.0,2,1,6.0,1,NaN,0.0,0.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,1.0,1,0,NaN,0,0,0,0,0,0
4,2018GQ0000251,6,2701,3,1,1013097,16,25,1,1.0,2.0,2,2,2.0,2.0,NaN,NaN,1.0,NaN,1,2,2,2,2,2,0.0,NaN,2.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,12.0,0.0,2,0.0,0.0,0.0,NaN,2.0,NaN,NaN,NaN,1,6.0,1,1,6.0,1,NaN,0.0,0.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
5,2018GQ0000390,6,2400,3,1,1013097,25,30,1,NaN,1.0,2,2,1.0,1.0,NaN,NaN,1.0,NaN,2,2,2,1,2,2,0.0,NaN,2.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,16.0,0.0,2,0.0,0.0,0.0,NaN,3.0,NaN,NaN,NaN,1,6.0,1,1,6.0,1,NaN,0.0,0.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
6,2018GQ0000510,6,400,3,1,1013097,18,66,1,NaN,1.0,2,2,1.0,1.0,NaN,NaN,1.0,NaN,2,2,1,2,2,2,0.0,NaN,2.0,2,2.0,1.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,19.0,0.0,2,0.0,15000.0,0.0,NaN,3.0,NaN,NaN,NaN,1,6.0,1,1,3.0,1,NaN,0.0,15000.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,1,0,0,0
7,2018GQ0000943,6,600,3,1,1013097,85,38,1,1.0,2.0,1,2,2.0,2.0,NaN,NaN,2.0,NaN,2,2,2,2,2,2,0.0,NaN,2.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,12.0,0.0,1,0.0,0.0,0.0,NaN,2.0,NaN,NaN,NaN,1,6.0,2,1,6.0,1,NaN,0.0,0.0,NaN,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
8,2018GQ0001008,6,2200,3,1,1013097,16,41,1,1.0,2.0,2,2,2.0,2.0,NaN,NaN,2.0,NaN,1,2,2,2,2,2,0.0,1.0,2.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,16.0,0.0,1,0.0,0.0,24000.0,40.0,1.0,4.0,1.0,NaN,2,1.0,1,1,6.0,1,NaN,24000.0,24000.0,184.0,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
10,2018GQ0001151,6,2200,3,1,1013097,92,65,1,NaN,1.0,2,2,1.0,1.0,NaN,NaN,1.0,NaN,2,2,1,1,2,2,0.0,NaN,2.0,1,2.0,1.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,19.0,0.0,1,0.0,12000.0,0.0,NaN,3.0,NaN,NaN,NaN,1,6.0,1,1,2.0,1,NaN,0.0,12000.0,NaN,2,0,0,1,0,1,0,0,NaN,NaN,NaN,1,0,NaN,0,0,1,0,0,0
11,2018GQ0001207,6,2400,3,1,1013097,31,79,1,NaN,2.0,1,2,1.0,1.0,NaN,NaN,1.0,NaN,2,2,1,1,2,2,0.0,NaN,2.0,2,2.0,1.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,13.0,0.0,2,0.0,9900.0,0.0,NaN,3.0,NaN,NaN,NaN,1,6.0,1,1,3.0,1,NaN,0.0,9900.0,NaN,2,0,0,1,0,1,0,0,NaN,NaN,NaN,1,0,NaN,0,0,1,0,0,0
12,2018GQ0001284,6,1303,3,1,1013097,16,57,4,NaN,1.0,1,2,1.0,1.0,2.0,1.0,1.0,NaN,2,2,2,2,2,2,0.0,NaN,2.0,4,2.0,1.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1400.0,0.0,0.0,1.0,19.0,0.0,1,0.0,0.0,0.0,NaN,3.0,NaN,2.0,1966.0,1,6.0,2,5,5.0,2,NaN,0.0,1400.0,10.0,8,0,0,0,0,1,1,0,NaN,NaN,12.0,3,0,NaN,0,0,0,0,0,0


## Create Veteran Only Dataset
---
Establishing a dataeset of only veterans will allow for detailed analysis of the veteran population and those that acknowledge being a veteran and yet do not enroll in the VA.

In [11]:
# Create seperate DataFrame of just veterans
vet_1 = pop[pop.MIL != 4]
vet_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249072 entries, 3 to 1648492
Data columns (total 88 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   SERIALNO   249072 non-null  object 
 1   DIVISION   249072 non-null  int64  
 2   PUMA       249072 non-null  int64  
 3   REGION     249072 non-null  int64  
 4   ST         249072 non-null  int64  
 5   ADJINC     249072 non-null  int64  
 6   PWGTP      249072 non-null  int64  
 7   AGEP       249072 non-null  int64  
 8   CIT        249072 non-null  int64  
 9   COW        146866 non-null  float64
 10  DDRS       249072 non-null  float64
 11  DEAR       249072 non-null  int64  
 12  DEYE       249072 non-null  int64  
 13  DOUT       249072 non-null  float64
 14  DPHY       249072 non-null  float64
 15  DRAT       50268 non-null   float64
 16  DRATX      249072 non-null  float64
 17  DREM       249072 non-null  float64
 18  ENG        19130 non-null   float64
 19  HINS1      249072 non-

In [12]:
#Save as seperate csv
vet_1.to_csv('../working_data/vets_1yr_compressed.csv', index=False)

## PUMs 5 Year Data Import (2013-2018)
---
Computer do not fail me now.

In [13]:
# Read in part 1 of 4
pop5A = pd.read_csv("../PUMs_data/psam_pusa2.csv")
pop5A.info()

/Users/joshkuehl/opt/anaconda3/envs/dsi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4723109 entries, 0 to 4723108
Columns: 286 entries, RT to PWGTP80
dtypes: float64(85), int64(197), object(4)
memory usage: 10.1+ GB


In [14]:
pop5A = column_remover(pop5A)
pop5A.head()

total time (s)= 38.13717579841614


,SERIALNO,DIVISION,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,CIT,COW,DDRS,DEAR,DEYE,DOUT,DPHY,DRAT,DRATX,DREM,ENG,HINS1,HINS2,HINS3,HINS4,HINS5,HINS6,INTP,JWRIP,LANX,MAR,MARHD,MARHT,MARHW,MIL,MLPA,MLPB,MLPCD,MLPE,MLPFG,MLPH,MLPI,MLPJ,MLPK,NWLK,OIP,PAP,RETP,SCH,SCHL,SEMP,SEX,SSIP,SSP,WAGP,WKHP,WKL,WKW,WRK,YOEP,DIS,ESR,HICOV,HISP,MSP,NATIVITY,OC,PERNP,PINCP,POVPIP,RAC1P,RACAIAN,RACASN,RACBLK,RACNH,RACNUM,RACSOR,RACWHT,SCIENGP,SCIENGRLP,VPS,WAOB,FCITP,FHINS5C,FHINS5P,FHINS6P,FMARHYP,FMILPP,FMILSP,FPUBCOVP
0,2014000000007,6,2400,3,1,1070673,7,49,1,6.0,2.0,2,2,2.0,2.0,NaN,NaN,2.0,NaN,2,1,2,2,2,2,0.0,1.0,2.0,1,2.0,1.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,16.0,45000.0,1,0.0,0.0,0.0,60.0,1.0,1.0,1.0,NaN,2,1.0,1,1,1.0,1,0.0,45000.0,45000.0,136.0,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
1,2014000000007,6,2400,3,1,1070673,8,59,1,NaN,2.0,2,2,2.0,1.0,NaN,NaN,2.0,NaN,2,2,2,2,2,2,0.0,NaN,2.0,1,2.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,1.0,1.0,0.0,2,0.0,0.0,0.0,NaN,3.0,NaN,2.0,NaN,1,6.0,2,1,1.0,1,0.0,0.0,0.0,136.0,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
2,2014000000007,6,2400,3,1,1070673,26,37,1,1.0,2.0,2,2,2.0,2.0,NaN,NaN,2.0,NaN,2,2,2,1,2,2,0.0,NaN,2.0,1,2.0,3.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,NaN,2.0,NaN,2.0,NaN,2,6.0,1,1,1.0,1,0.0,0.0,0.0,136.0,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,1,0,0,0
5,2014000000007,6,2400,3,1,1070673,16,34,1,NaN,2.0,2,2,2.0,2.0,NaN,NaN,2.0,NaN,2,2,2,1,2,2,0.0,NaN,2.0,1,2.0,1.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,1.0,17.0,0.0,2,0.0,0.0,0.0,NaN,3.0,NaN,2.0,NaN,2,6.0,1,1,1.0,1,0.0,0.0,0.0,136.0,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
6,2014000000053,6,700,3,1,1070673,51,51,1,1.0,2.0,2,2,2.0,2.0,NaN,2.0,2.0,NaN,1,2,2,2,2,2,0.0,1.0,2.0,1,2.0,1.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,16.0,0.0,1,0.0,0.0,24000.0,40.0,1.0,1.0,1.0,NaN,2,1.0,1,1,1.0,1,0.0,24000.0,24000.0,288.0,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0


In [15]:
# Read in part 2 of 4
pop5B = pd.read_csv("../PUMs_data/psam_pusb2.csv")
pop5B.info()

/Users/joshkuehl/opt/anaconda3/envs/dsi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3417358 entries, 0 to 3417357
Columns: 286 entries, RT to PWGTP80
dtypes: float64(85), int64(197), object(4)
memory usage: 7.3+ GB


In [16]:
pop5B = column_remover(pop5B)
pop5B.info()

total time (s)= 24.77752184867859
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2611374 entries, 0 to 3417357
Data columns (total 88 columns):
 #   Column     Dtype  
---  ------     -----  
 0   SERIALNO   object 
 1   DIVISION   int64  
 2   PUMA       int64  
 3   REGION     int64  
 4   ST         int64  
 5   ADJINC     int64  
 6   PWGTP      int64  
 7   AGEP       int64  
 8   CIT        int64  
 9   COW        float64
 10  DDRS       float64
 11  DEAR       int64  
 12  DEYE       int64  
 13  DOUT       float64
 14  DPHY       float64
 15  DRAT       float64
 16  DRATX      float64
 17  DREM       float64
 18  ENG        float64
 19  HINS1      int64  
 20  HINS2      int64  
 21  HINS3      int64  
 22  HINS4      int64  
 23  HINS5      int64  
 24  HINS6      int64  
 25  INTP       float64
 26  JWRIP      float64
 27  LANX       float64
 28  MAR        int64  
 29  MARHD      float64
 30  MARHT      float64
 31  MARHW      float64
 32  MIL        float64
 33  MLPA    

In [17]:
# Read in part 3 of 4
pop5C = pd.read_csv("../PUMs_data/psam_pusc.csv")
pop5C.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3698249 entries, 0 to 3698248
Columns: 286 entries, RT to PWGTP80
dtypes: float64(85), int64(197), object(4)
memory usage: 7.9+ GB


In [18]:
pop5C = column_remover(pop5C)
pop5C.info()

total time (s)= 32.08351397514343
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2838749 entries, 0 to 3698246
Data columns (total 88 columns):
 #   Column     Dtype  
---  ------     -----  
 0   SERIALNO   object 
 1   DIVISION   int64  
 2   PUMA       int64  
 3   REGION     int64  
 4   ST         int64  
 5   ADJINC     int64  
 6   PWGTP      int64  
 7   AGEP       int64  
 8   CIT        int64  
 9   COW        float64
 10  DDRS       float64
 11  DEAR       int64  
 12  DEYE       int64  
 13  DOUT       float64
 14  DPHY       float64
 15  DRAT       float64
 16  DRATX      float64
 17  DREM       float64
 18  ENG        float64
 19  HINS1      int64  
 20  HINS2      int64  
 21  HINS3      int64  
 22  HINS4      int64  
 23  HINS5      int64  
 24  HINS6      int64  
 25  INTP       float64
 26  JWRIP      float64
 27  LANX       float64
 28  MAR        int64  
 29  MARHD      float64
 30  MARHT      float64
 31  MARHW      float64
 32  MIL        float64
 33  MLPA    

In [19]:
# Read in part 4 of 4
pop5D = pd.read_csv("../PUMs_data/psam_pusd.csv")
pop5D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4001965 entries, 0 to 4001964
Columns: 286 entries, RT to PWGTP80
dtypes: float64(85), int64(197), object(4)
memory usage: 8.5+ GB


In [20]:
pop5D = column_remover(pop5D)
pop5D.head()

total time (s)= 28.442046403884888


,SERIALNO,DIVISION,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,CIT,COW,DDRS,DEAR,DEYE,DOUT,DPHY,DRAT,DRATX,DREM,ENG,HINS1,HINS2,HINS3,HINS4,HINS5,HINS6,INTP,JWRIP,LANX,MAR,MARHD,MARHT,MARHW,MIL,MLPA,MLPB,MLPCD,MLPE,MLPFG,MLPH,MLPI,MLPJ,MLPK,NWLK,OIP,PAP,RETP,SCH,SCHL,SEMP,SEX,SSIP,SSP,WAGP,WKHP,WKL,WKW,WRK,YOEP,DIS,ESR,HICOV,HISP,MSP,NATIVITY,OC,PERNP,PINCP,POVPIP,RAC1P,RACAIAN,RACASN,RACBLK,RACNH,RACNUM,RACSOR,RACWHT,SCIENGP,SCIENGRLP,VPS,WAOB,FCITP,FHINS5C,FHINS5P,FHINS6P,FMARHYP,FMILPP,FMILSP,FPUBCOVP
0,2014000000002,2,1000,1,42,1070673,5,56,1,4.0,2.0,2,2,2.0,2.0,NaN,NaN,2.0,NaN,2,1,2,2,2,2,0.0,1.0,2.0,4,2.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5400.0,0.0,0.0,1.0,16.0,0.0,2,0.0,0.0,28000.0,40.0,1.0,1.0,1.0,NaN,2,1.0,1,1,5.0,1,0.0,28000.0,33400.0,198.0,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
1,2014000000002,2,1000,1,42,1070673,6,28,1,1.0,2.0,2,2,2.0,2.0,NaN,NaN,2.0,NaN,1,2,2,2,2,2,0.0,1.0,2.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,19.0,0.0,1,0.0,0.0,14000.0,30.0,1.0,1.0,NaN,NaN,2,1.0,1,1,6.0,1,0.0,14000.0,14000.0,198.0,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,1,NaN,1,1,0,0,1,1
4,2014000000002,2,1000,1,42,1070673,10,23,1,1.0,2.0,2,2,2.0,2.0,NaN,NaN,2.0,NaN,2,2,2,2,2,2,0.0,1.0,2.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,3.0,21.0,0.0,2,0.0,0.0,7500.0,40.0,1.0,4.0,NaN,NaN,2,1.0,2,1,6.0,1,0.0,7500.0,7500.0,61.0,1,0,0,0,0,1,0,1,1.0,2.0,NaN,1,1,NaN,1,1,0,0,1,1
5,2014000000038,2,701,1,42,1070673,9,40,1,1.0,2.0,2,2,2.0,2.0,NaN,NaN,2.0,NaN,1,2,2,2,2,2,0.0,1.0,2.0,4,2.0,1.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,2.0,19.0,0.0,2,0.0,0.0,11500.0,40.0,1.0,1.0,1.0,NaN,2,1.0,1,1,5.0,1,0.0,11500.0,11500.0,71.0,1,0,0,0,0,1,0,1,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0
7,2014000000039,2,3204,1,42,1070673,15,20,1,1.0,2.0,2,2,2.0,2.0,NaN,NaN,2.0,1.0,1,2,2,2,2,2,0.0,NaN,1.0,5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,3.0,16.0,0.0,2,0.0,0.0,5000.0,15.0,1.0,1.0,1.0,NaN,2,1.0,1,2,6.0,1,0.0,5000.0,5000.0,NaN,9,0,0,0,0,2,1,0,NaN,NaN,NaN,1,0,NaN,0,0,0,0,0,0


In [21]:
# Combine Person Data A and B
pop5 = pop5D.append([pop5A, pop5B, pop5C])
pop5.to_csv('../working_data/us_pop_5yr_compressed.csv', index=False)

In [22]:
# Create seperate DataFrame of just veterans
vet_5 = pop5[pop5.MIL != 4]
vet_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1273939 entries, 13 to 3698212
Data columns (total 88 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   SERIALNO   1273939 non-null  object 
 1   DIVISION   1273939 non-null  int64  
 2   PUMA       1273939 non-null  int64  
 3   REGION     1273939 non-null  int64  
 4   ST         1273939 non-null  int64  
 5   ADJINC     1273939 non-null  int64  
 6   PWGTP      1273939 non-null  int64  
 7   AGEP       1273939 non-null  int64  
 8   CIT        1273939 non-null  int64  
 9   COW        772087 non-null   float64
 10  DDRS       1273939 non-null  float64
 11  DEAR       1273939 non-null  int64  
 12  DEYE       1273939 non-null  int64  
 13  DOUT       1273939 non-null  float64
 14  DPHY       1273939 non-null  float64
 15  DRAT       234229 non-null   float64
 16  DRATX      1273939 non-null  float64
 17  DREM       1273939 non-null  float64
 18  ENG        90946 non-null    float64
 19 

In [23]:
#Save as seperate csv
vet_5.to_csv('../working_data/vets_5yr_compressed.csv', index=False)

In [24]:
print("All files compressed and saved!")

All files compressed and saved!
